In [1]:
import importlib
import data
from gurobipy import *

# get data from excel
dataframes = data.get_all_pandas()

LOADING DATA: 8it [00:37,  3.17s/it]                       

In [2]:
from openpyxl import Workbook
from datetime import datetime
from utils import min_to_jour

def show_solution(model):
    # print(var_dict)
    print("Objectif minimal: ", m.objVal)
    #build excel of tasks
    workbook = Workbook()
    sheet = workbook.active
    sillons_df = dataframes["sillons_df"]
    taches_df = dataframes["taches_df"]
    machines_df = dataframes["machines_df"]

    sheet["A1"] = "Id_tache"
    sheet["B1"] = "Type de tache"
    sheet["C1"] = "Jour"
    sheet["D1"] = "HDébut"
    sheet["E1"] = "Durée"
    sheet["F1"] = "Sillon"
    sheet["G1"] = "Minute"
    row = 2
    for tache in var_dict.keys():
        for sillon_id in var_dict[tache].keys():
            sheet[f"B{row}"] = tache
            sheet[f"A{row}"] = tache + "_" + sillon_id
            sheet[f"C{row}"] = min_to_jour(var_dict[tache][sillon_id].X)[0]
            sheet[f"D{row}"] = min_to_jour(var_dict[tache][sillon_id].X)[1]
            if tache in list(machines_df["Machine"]):
                sheet[f"E{row}"] = machines_df[machines_df["Machine"] == tache]["Duree "].iloc[0]
            elif tache in list(taches_df["Type de tache humaine"]): 
                sheet[f"E{row}"] = taches_df[taches_df["Type de tache humaine"] == tache]["Durée"].iloc[0]
            sheet[f"F{row}"] = sillon_id
            sheet[f"G{row}"] = var_dict[tache][sillon_id].X
            row += 1

    currentDateAndTime = datetime.now().strftime("%d-%m_%H-%M-%S")
    workbook.save(filename=f"results/{data.NAME}-{currentDateAndTime}.xlsx")
# show_solution(m)


In [3]:
import variables
import contraintes
import contraintesJ2
import importlib
import utils
importlib.reload(variables) #to remive in final .py

importlib.reload(utils)

<module 'utils' from 'c:\\Users\\remgo\\Desktop\\ProjetST7\\git\\utils.py'>

In [5]:

#to remive in final .py
importlib.reload(utils)
importlib.reload(variables)
importlib.reload(contraintes) 
importlib.reload(contraintesJ2)
# create model
m = Model()

# generate variables (integers)
var_dict = variables.generate_variablesJ1(m, dataframes)
links_dict = variables.generate_variablesJ2(m,var_dict,dataframes)
# generate variables
B, OCCUPATIONS = contraintes.generate_contraintes(m, dataframes, var_dict)
contraintesJ2.generate_contraintes(m, dataframes, var_dict, links_dict)

final_train_task = dataframes["sillons_df"].copy()
final_train_task = final_train_task[final_train_task["LDEP"]=="WPY"]
final_train_task = final_train_task[final_train_task["JARR"]==final_train_task["JARR"].max()]
# final_train_task = final_train_task[final_train_task["HDEP"]==final_train_task["HDEP"].max()]["train_id"].iloc[0]
# final_train_task = var_dict["essai de frein départ"][final_train_task]
final_train_task = quicksum([var_dict["essai de frein départ"][temp] for temp in list(final_train_task["train_id"])])
print(final_train_task)
m.setObjective(final_train_task, GRB.MINIMIZE)
# autre idée : maximiser la moyenne de l'ecart entre frein de depart et HDEP
# aute idee minimiser occupation
m.update()
# m.printStats()
# m.write("genconstr.mps")
# m.write("test.lp")
m.optimize()
# m.write("test.lp")

show_solution(m)

Initialising variables:   0%|          | 0/217 [00:00<?, ?it/s]

Link variables:   0%|          | 0/5 [00:00<?, ?it/s]

BUILDING CONSTRAINTS:   0%|          | 0/8 [00:00<?, ?it/s]

BUILDING CONSTRAINTS FOR J2:   0%|          | 0/4 [00:00<?, ?it/s]

KeyError: ('arrivée Reception', '4126409/08/202200:0022:3309/08/2022', 'roulement_reception', 1, 1, 1)

Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

In [ ]:
# tests 
importlib.reload(contraintes)
importlib.reload(contraintesJ2)
importlib.reload(variables)
importlib.reload(utils)
# create model
m = Model()
# generate variables (integers)
var_dict = variables.generate_variablesJ1(m, dataframes)
links_dict = variables.generate_variablesJ2(m,var_dict,dataframes)
# generate constraints 
# contraintes.generate_contraintes(m, dataframes, var_dict)
contraintesJ2.generate_contraintes(m, dataframes, var_dict, links_dict)

# deposer solution jalon1  et jalon 2 maybe # new objectif ??
# add excels for jalon 2 in show solution

In [ ]:
show_solution(m)